In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore') #忽视警告

plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文字体设置-黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题

import jieba
import jieba.analyse
from wordcloud import WordCloud


# 读取数据

In [22]:
df = pd.read_csv("douban_queen_reviews.csv")

# 分词

In [23]:
#加载外部分词字典
jieba.load_userdict("userdict.txt")

# 使用jieba进行中文分词
def chinese_segmentation(text):
    words = jieba.cut(text, cut_all=False, HMM=True)
    return " ".join(words)

In [24]:
# 先处理缺失值，将缺失值替换为一个空字符串或其他合适的值
df['content'].fillna('', inplace=True)
# 对DataFrame的'content'列应用中文分词
df['content_segmented'] = df['content'].apply(chinese_segmentation)

In [25]:
df['content_segmented']

0      前 几天 有 一句 十分 歹毒 的 骂人 话 冲 上 了 热 搜 ， 出处 是 正在 播出 ...
1      一不小心 一口气 追剧 到   12   集 。 不说 一些 浮夸 的 剧情 ， 剧情 来自...
2      今晚 看 《 新闻 女王 》 第   15   集 ， 很 有意思 。 文慧心 回到 家 ，...
3      “ 找个 男人 嫁 了 吧 ” 比 以前   TVB   宫斗剧 扇 耳光 更 “ 狠 ” ...
4      在 正式 看 这部 剧 之前 ， 光听 名字 就 觉得 像 个 烂剧 ， “ 女王 ” 两个...
                             ...                        
176    很久没 见 一部 剧 值得 给 长评 了 ， 这部 大约 是 值得 的 。 只是 等 了 这...
177    文   \   榴花 照图   \   源自 网络 ， 侵删 致歉 ------------...
178    《 新闻 女王 》 是 用 宫斗剧 的 打法 拍 现代 职场 剧 。 第十集 ， 文慧心 成...
179    1 .   新闻 主播 英语 「 Anchor 」 \n 原 解作 固定 船身 的 锚 \n...
180    今年 我 觉得 最 魔幻现实主义 的 事 发生 了 ， 当内 娱在 播放 《 以爱为 营 》...
Name: content_segmented, Length: 181, dtype: object

# 同义词替换

In [11]:
# 加载同义词文件
synonym_file_path = 'synonym.txt'
synonym_dict = {}

# 读取同义词文件，创建同义词典，将同义词映射到主词
with open(synonym_file_path, 'r', encoding='utf-8') as file:
    for line in file:
        parts = line.strip().split()
        if len(parts) > 1:
            main_word = parts[0]
            synonyms = parts[1:]
            for synonym in synonyms:
                synonym_dict[synonym] = main_word

# 替换同义词的函数（针对已分词的情况）
def replace_synonyms(words):
    # 使用同义词典替换同义词为主词
    replaced_words = [synonym_dict.get(word, word) for word in words]

    # 将分词结果拼接成字符串
    return " ".join(replaced_words)

# 对 'content_segmented' 列应用替换同义词的函数
df['content_segmented'] = df['content_segmented'].apply(lambda x: replace_synonyms(x.split()))

# 显示替换同义词后的 DataFrame
print(df['content_segmented'])


0      前 几天 有 一句 十分 歹毒 的 骂人 话 冲 上 了 热 搜 ， 出处 是 正在 播出 ...
1      一不小心 一口气 追剧 到 12 集 。 不说 一些 浮夸 的 剧情 ， 剧情 来自 生活 ...
2      今晚 看 《 新闻 女王 》 第 15 集 ， 很 有意思 。 文慧心 回到 家 ， 发现 ...
3      “ 找个 男人 嫁 了 吧 ” 比 以前 TVB 宫斗剧 扇 耳光 更 “ 狠 ” 啊 ， ...
4      在 正式 看 这部 剧 之前 ， 光听 名字 就 觉得 像 个 烂剧 ， “ 女王 ” 两个...
                             ...                        
176    很久没 见 一部 剧 值得 给 长评 了 ， 这部 大约 是 值得 的 。 只是 等 了 这...
177    文 \ 榴花 照图 \ 源自 网络 ， 侵删 致歉 --------------------...
178    《 新闻 女王 》 是 用 宫斗剧 的 打法 拍 现代 职场 剧 。 第十集 ， 文慧心 成...
179    1 . 新闻 主播 英语 「 Anchor 」 原 解作 固定 船身 的 锚 引申 稳住 新...
180    今年 我 觉得 最 魔幻现实主义 的 事 发生 了 ， 当内 娱在 播放 《 以爱为 营 》...
Name: content_segmented, Length: 181, dtype: object


# 哪些影视作品被quote到

In [9]:
import re
import pandas as pd

# 提取包含在《》中的字符串的函数
def extract_titles(text):
    # 使用正则表达式提取《》中的字符串
    matches = re.findall(r'《(.*?)》', text)
    return matches

# 创建新的 DataFrame df_movie
df_movie = pd.DataFrame(columns=['Movie Title', 'Original Title'])


In [10]:
df_movie


,Movie Title,Original Title


In [42]:
df_movie.to_excel("df_movie.xlsx")

In [8]:
# 使用 drop_duplicates 方法去除同一 Original Title 下的重复 Movie Title
df_unique_titles = df_movie[['Movie Title', 'Original Title']].drop_duplicates()

# 统计 Movie Title 的频率
title_frequency = df_unique_titles['Movie Title'].value_counts()

# 打印结果或进行其他操作
title_frequency.head(20)

Series([], Name: count, dtype: int64)

# 词频分析

In [12]:
# 将分好词的文本拆分成词语列表
df['content_segmented_list'] = df['content_segmented'].str.split()

# 使用explode将词语列表展开为单独的行
df_tokens = df.explode('content_segmented_list')

# 重新设置索引
df_tokens = df_tokens.reset_index(drop=True)

# 打印结果
df_tokens.content_segmented_list

0          前
1         几天
2          有
3         一句
4         十分
          ..
100632    现实
100633     的
100634     了
100635     吗
100636     ？
Name: content_segmented_list, Length: 100637, dtype: object

In [13]:
# 词频统计
df_tokens["content_segmented_list"].value_counts()

content_segmented_list
，      8872
的      5484
。      3175
是      1684
了      1236
       ... 
散心        1
常         1
闭店        1
店老板       1
几十个       1
Name: count, Length: 12898, dtype: int64

In [14]:
#导出数据
df_tokens["content_segmented_list"].value_counts().to_excel("reviews_freq.xlsx")

# LDA分析

In [26]:
# 导入所需的库
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel
from gensim.parsing.preprocessing import preprocess_string
from nltk.corpus import stopwords
from gensim import matutils
import gensim
import matplotlib.pyplot as plt
from gensim.models import CoherenceModel

# 预处理文本数据
# 在这里，我们使用gensim的预处理函数，包括分词、去除停用词等

# 步骤1：从外部文件读取停用词
with open("stopwords.txt", "r", encoding="utf-8") as file:
    stop_words = file.read().splitlines()

# 步骤2：将每一行的空格分隔单词转换为单词列表
df['content_segmented_list'] = df['content_segmented'].apply(lambda x: x.split())

# 步骤3：过滤停用词
df['content_segmented_list'] = df['content_segmented_list'].apply(lambda x: [word for word in x if word not in stop_words])

# 创建字典和语料库
dictionary = corpora.Dictionary(df['content_segmented_list'])
corpus = [dictionary.doc2bow(text) for text in df['content_segmented_list']]


In [28]:
# 定义计算困惑度的函数
def compute_perplexity(corpus, model):
    return model.log_perplexity(corpus)

# 定义计算主题一致性的函数
def compute_coherence(corpus, model, dictionary, texts):
    coherence_model = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
    return coherence_model.get_coherence()

# 定义一个函数，用于绘制困惑度和主题一致性的图表
def plot_metrics(num_topics_list, perplexity_values, coherence_values):
    fig, ax1 = plt.subplots()

    ax1.set_xlabel('Number of Topics')
    ax1.set_ylabel('Perplexity', color='tab:blue')
    ax1.plot(num_topics_list, perplexity_values, color='tab:blue', marker='o')
    ax1.tick_params(axis='y', labelcolor='tab:blue')

    ax2 = ax1.twinx()
    ax2.set_ylabel('Coherence', color='tab:red')
    ax2.plot(num_topics_list, coherence_values, color='tab:red', marker='o')
    ax2.tick_params(axis='y', labelcolor='tab:red')

    fig.tight_layout()
    plt.title('LDA Model Evaluation Metrics')
    plt.show()

# 定义一个函数，用于训练LDA模型并计算评估指标
def train_lda_model_and_evaluate(corpus, dictionary, texts, num_topics_list):
    perplexity_values = []
    coherence_values = []

    for num_topics in num_topics_list:
        # 训练LDA模型
        lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

        # 计算困惑度
        perplexity = compute_perplexity(corpus, lda_model)
        perplexity_values.append(perplexity)

        # 计算主题一致性
        coherence = compute_coherence(corpus, lda_model, dictionary, texts)
        coherence_values.append(coherence)

    # 绘制图表
    plot_metrics(num_topics_list, perplexity_values, coherence_values)

# 选择要尝试的主题数量范围
num_topics_list = list(range(1, 11))

# 训练LDA模型并评估
train_lda_model_and_evaluate(corpus, dictionary, df['content_segmented_list'], num_topics_list)
